In [1]:
import ast

import intake
from distributed import Client
from ncar_jobqueue import NCARCluster

from ecgtools import Builder
from ecgtools.parsers.cesm import parse_cesm_history

In [2]:
cluster = NCARCluster()
client = Client(cluster)
cluster

/glade/work/abanihi/opt/miniconda/envs/playground/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34697 instead
  warnings.warn(


In [3]:
b = Builder(
    "/glade/scratch/kristenk/archive/",
    depth=2,
    parser=parse_cesm_history,
    exclude_patterns=["*/tseries/*", "*/rest/*"],
)

In [4]:
b = b.build(
    path_column='path',
    variable_column='variables',
    data_format="netcdf",
    groupby_attrs=["component", "stream", "case"],
    aggregations=[
        {
            "type": "join_existing",
            "attribute_name": "date",
            "options": {"dim": "time", "coords": "minimal", "compat": "override"},
        }
    ],
)

b.save("cesm-hist-test.csv.gz")

Getting list of directories...

Getting list of files...

Batching `_get_filelist()` dask.delayed calls...

Harvesting attributes/metadata from files...

Unable to parse 14 assets. For a full list of these assets, see invalid_assets.txt file

Saved catalog location: <_io.TextIOWrapper name='cesm-hist-test.json' mode='w' 
encoding='UTF-8'> and cesm-hist-test.csv.gz

Builder(root_path=PosixPath('/glade/scratch/kristenk/archive'), extension='*.nc', depth=2, exclude_patterns=['*/tseries/*', '*/rest/*'], parser=<function parse_cesm_history at 0x2b7634466790>, lazy=True, nbatches=25)

In [5]:
b.df.head()

,component,stream,case,frequency,date,variables,path
0,ocn,pop.h.ecosys,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,nday1.0053-01-01,"[photoC_sp_zint_2, photoC_diat_zint_2, photoC_...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
1,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0046-08,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
2,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0061-07,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
3,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0042-09,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
4,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0043-11,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...


In [9]:
col = intake.open_esm_datastore(
    "cesm-hist-test.json", csv_kwargs={"converters": {"variables": ast.literal_eval}}, sep="/"
)
col

,unique
component,2
stream,5
case,14
frequency,3
date,2258
variables,691
path,19859


In [10]:
col.df.head()

,component,stream,case,frequency,date,variables,path
0,ocn,pop.h.ecosys,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,nday1.0053-01-01,"[photoC_sp_zint_2, photoC_diat_zint_2, photoC_...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
1,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0046-08,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
2,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0061-07,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
3,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0042-09,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...
4,ocn,pop.h,g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p...,month_1,0043-11,"[UVEL, UVEL2, VVEL, VVEL2, TEMP, dTEMP_POS_2D,...",/glade/scratch/kristenk/archive/g.e22a06.G1850...


In [11]:
cat = col.search(variables=["UVEL"], case=["g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p4z.003"])
cat

,unique
component,1
stream,1
case,1
frequency,1
date,288
variables,498
path,288


In [12]:
dsets = cat.to_dataset_dict(cdf_kwargs={'use_cftime': True, 'chunks': {}})
dsets


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component/stream/case'


{'ocn/pop.h/g.e22a06.g1850ecoiaf_jra_phys_dev.tl319_g17.4p4z.003': <xarray.Dataset>
 Dimensions:  (nlat: 384, nlon: 320, time: 288, z_t: 60)
 Coordinates:
   * time     (time) object 0003-06-01 00:00:00 ... 0007-04-01 00:00:00
   * z_t      (z_t) float32 500.0 1.5e+03 2.5e+03 ... 5.125e+05 5.375e+05
     ULONG    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
     ULAT     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
     TLONG    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
     TLAT     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
 Dimensions without coordinates: nlat, nlon
 Data variables:
     UVEL     (time, z_t, nlat, nlon) float32 dask.array<chunksize=(1, 60, 384, 320), meta=np.ndarray>
 Attributes:
     model_doi_url:           https://doi.org/10.5065/D67H1H0V
     intake_esm_varname:      ['UVEL']
     title:                   g.e22a06.G1850ECOIAF_JRA_PHYS_DEV.TL319_g17.4p4z..